In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
import re
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [90]:
def evaluate(model):
    print(np.bincount(np.where((model.predict(X_test)-y_test) !=0, 0, 1))/len(y_test))

In [91]:
data = pd.read_csv('data.csv')

In [92]:
print(data.shape)
data.head()

(2550, 2)


,name,currency
0,cia exposed by frontline role along afghanista...,1.616810
1,somali charged with murder attempt on muhammad...,1.620700
2,ethiopia firm recycling tyres into shoes does ...,1.616800
3,militants behind embassy threats killed yemen...,1.610591
4,more body searches sniffer dogs and luggage i...,1.600088


In [93]:
Y = data.as_matrix(columns=['currency']).reshape(-1)

In [94]:
Y = np.gradient(Y)
Y

array([  3.89000000e-03,  -5.00000000e-06,  -5.05450000e-03, ...,
         5.83950000e-03,   4.65100000e-03,  -5.69000000e-04])

In [95]:
# test = np.array([0,1,2,3,4,5,5,4])
# np.gradient(test)

In [96]:
Y = np.where(Y > 0, 1, 0)
Y[:10]

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [97]:
vectorizer = TfidfVectorizer(min_df=1)

In [98]:
X = vectorizer.fit_transform(data['name'].tolist())

In [99]:
# temp = [X]
# for i in range(5):
#     temp.append(X.power(i))
# X = hstack(temp)

In [100]:
X

<2550x29958 sparse matrix of type '<class 'numpy.float64'>'
	with 531411 stored elements in Compressed Sparse Row format>

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
print(X_train.shape)
print(X_test.shape)

(2040, 29958)
(510, 29958)


In [102]:
# model = LogisticRegression(fit_intercept=True,
#                            class_weight = 'balanced',
#                            n_jobs=-1,
#                            max_iter=200, 
#                            solver='lbfgs',  
#                            verbose=1, 
#                            tol=1e-10)
# model.fit(X_train, y_train)
# model.score(X_test, y_test)

In [103]:
model = NuSVC(kernel='rbf',
                class_weight = 'balanced',
                degree = 7,
                verbose=0)
recall = cross_val_score(model, X, Y, cv=5, n_jobs = -1, pre_dispatch = '2*n_jobs', scoring='recall')
predicted = cross_val_predict(model, X, Y, cv=5, n_jobs = -1, pre_dispatch = '2*n_jobs')
precision = cross_val_score(model, X, Y, cv=5, n_jobs = -1, pre_dispatch = '2*n_jobs', scoring='precision')
print('precision:',np.mean(precision))
print('recall:', np.mean(recall))

precision: 0.510641295479
recall: 0.958301158301


In [104]:
predicted = cross_val_predict(model, X, Y, cv=5, n_jobs = -1, pre_dispatch = '2*n_jobs')

In [105]:
model.fit(X_train, y_train)
print(confusion_matrix(y_test, model.predict(X_test), labels=[0,1]))
target_names = ['down','up']
print(classification_report(y_test, model.predict(X_test), target_names=target_names))

[[ 76 172]
 [ 88 174]]
             precision    recall  f1-score   support

       down       0.46      0.31      0.37       248
         up       0.50      0.66      0.57       262

avg / total       0.48      0.49      0.47       510



In [106]:
# def benchmark(clf_class, params, name):
#     print("parameters:", params)
#     t0 = time()
#     clf = clf_class(**params).fit(X_train, y_train)
#     print("done in %fs" % (time() - t0))

#     if hasattr(clf, 'coef_'):
#         print("Percentage of non zeros coef: %f"
#               % (np.mean(clf.coef_ != 0) * 100))
#     print("Predicting the outcomes of the testing set")
#     t0 = time()
#     pred = clf.predict(X_test)
#     print("done in %fs" % (time() - t0))

#     print("Classification report on test set for classifier:")
#     print(clf)
#     print()
#     print(classification_report(y_test, pred,
#                                 target_names=news_test.target_names))

#     cm = confusion_matrix(y_test, pred)
#     print("Confusion matrix:")
#     print(cm)

#     # Show confusion matrix
#     plt.matshow(cm)
#     plt.title('Confusion matrix of the %s classifier' % name)
#     plt.colorbar()


# print("Testbenching a linear classifier...")
# parameters = {
#     'loss': 'hinge',
#     'penalty': 'l2',
#     'n_iter': 50,
#     'alpha': 0.00001,
#     'fit_intercept': True,
# }

# benchmark(SGDClassifier, parameters, 'SGD')

# print("Testbenching a MultinomialNB classifier...")
# parameters = {'alpha': 0.01}

# benchmark(MultinomialNB, parameters, 'MultinomialNB')

# plt.show()